## Fase 1

In [1]:
import pandas as pd

In [5]:
plaza = pd.read_csv('people.csv',delimiter=";")
plaza.loc[plaza.Prof=='Guardia','Num'].sum()

44

## Fase 2

In [6]:
import ftplib
ftp = ftplib.FTP('f31-preview.runhosting.com','4009006_DATOS','Rafa9999')
filename = 'books.xml'
ftp.retrbinary("RETR "+filename,open(filename,'wb').write)

'226 Transfer complete'

In [7]:
ftp.quit()

'221 Goodbye.'

In [8]:
import xml.etree.ElementTree as ET
tree = ET.parse('books.xml')

In [9]:
lst = tree.findall('book')
len(lst)

100

In [10]:
lst[43].find('link').text

'http://johnstoniatexts.x10host.com/homer/iliad1html.html'

## Fase 3

In [11]:
import urllib.request
from bs4 import BeautifulSoup
url = 'http://johnstoniatexts.x10host.com/homer/iliad1html.html'
html = urllib.request.urlopen(url)
soup = BeautifulSoup(html)

tags = soup.find_all('span',class_="auto-style21")
texto = ''

for tag in tags:
    try: texto = texto + tag.text
    except:None
        
contadores = dict() # Creo un diccionario para almacenar las palabras y contar las veces que aparece
texto = texto.replace(',','').replace('.','').replace(':','').replace('?','').replace('(','').replace(')','').replace('','') # Elimino signos de (puntos, comas, etc.)
palabras = texto.lower().split() # Paso todas las palabras a minúsculas (lower) y las corto palabra a palabra (split)

for palabra in palabras:
     if len(palabra)==6:
        contadores[palabra] = contadores.get(palabra,0) + 1 
        
df = pd.DataFrame(list(contadores.items()), columns=['palabra', 'contador']).sort_values('contador', ascending=False )
df#df.loc[df.palabra.str.contains('pollo')]

,palabra,contador
0,apollo,17
9,sacred,9
42,before,8
78,should,8
2,thetis,8
...,...,...
133,speaks,1
134,gorges,1
135,notson,1
136,ripped,1


## Fase 4

In [12]:
import sqlite3

In [13]:
conn = sqlite3.connect('tienda.sqlite')

In [14]:
pd.read_sql('SELECT name FROM sqlite_master WHERE type="table"',conn)

,name
0,Employee
1,Category
2,Customer
3,Shipper
4,Supplier
5,Order
6,Product
7,OrderDetail
8,CustomerCustomerDemo
9,CustomerDemographic


In [16]:
pd.read_sql('SELECT * FROM Product',conn)

,Id,ProductName,SupplierId,CategoryId,QuantityPerUnit,UnitPrice,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued
0,1,Chai,1,1,10 boxes x 20 bags,18.00,39,0,10,0
1,2,Chang,1,1,24 - 12 oz bottles,19.00,17,40,25,0
2,3,Aniseed Syrup,1,2,12 - 550 ml bottles,10.00,13,70,25,0
3,4,Chef Anton's Cajun Seasoning,2,2,48 - 6 oz jars,22.00,53,0,0,0
4,5,Chef Anton's Gumbo Mix,2,2,36 boxes,21.35,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
72,73,Röd Kaviar,17,8,24 - 150 g jars,15.00,101,0,5,0
73,74,Longlife Tofu,4,7,5 kg pkg.,10.00,4,20,5,0
74,75,Rhönbräu Klosterbier,12,1,24 - 0.5 l bottles,7.75,125,0,25,0
75,76,Lakkalikööri,23,1,500 ml,18.00,57,0,20,0


In [21]:
pd.read_sql('SELECT * FROM OrderDetail',conn)

,Id,OrderId,ProductId,UnitPrice,Quantity,Discount
0,10248/11,10248,11,14.00,12,0.00
1,10248/42,10248,42,9.80,10,0.00
2,10248/72,10248,72,34.80,5,0.00
3,10249/14,10249,14,18.60,9,0.00
4,10249/51,10249,51,42.40,40,0.00
...,...,...,...,...,...,...
2150,11077/64,11077,64,33.25,2,0.03
2151,11077/66,11077,66,17.00,1,0.00
2152,11077/73,11077,73,15.00,2,0.01
2153,11077/75,11077,75,7.75,4,0.00


In [22]:
query = '''
SELECT ProductName, SUM(Quantity) AS Total
FROM OrderDetail det
JOIN Product pro
ON det.ProductId = pro.Id
GROUP BY ProductName
ORDER BY Total DESC
'''

pd.read_sql(query,conn)

,ProductName,Total
0,Camembert Pierrot,1577
1,Raclette Courdavault,1496
2,Gorgonzola Telino,1397
3,Gnocchi di nonna Alice,1263
4,Pavlova,1158
...,...,...
72,Laughing Lumberjack Lager,184
73,Chocolade,138
74,Gravad lax,125
75,Genen Shouyu,122


## Fase 5

In [36]:
import urllib.request
endpoint = 'https://api.edamam.com/api/recipes/v2'
url = endpoint + '?q=camembert&app_id='+api_id+'&app_key='+api_key+'&type=public'
print(url)
data = urllib.request.urlopen(url).read().decode()
js = json.loads(data)
print(js)

https://api.edamam.com/api/recipes/v2?q=camembert&app_id=0ceaaf83&app_key=93ff26f92c228dbafa3d7d2f9b789a39&type=public
{'from': 1, 'to': 20, 'count': 449, '_links': {'next': {'href': 'https://api.edamam.com/api/recipes/v2?q=camembert&app_key=93ff26f92c228dbafa3d7d2f9b789a39&_cont=CHcVQBtNNQphDmgVQntAEX4BYlRtDQUHQ2JHBWcSalJ0AQAOUXlSVTYTZQQhAVADRjQUBWMUZ1MgBAEOQG0VBWYQYVN6VQMVLnlSVSBMPkd5BgMbUSYRVTdgMgksRlpSAAcRXTVGcV84SU4%3D&type=public&app_id=0ceaaf83', 'title': 'Next page'}}, 'hits': [{'recipe': {'uri': 'http://www.edamam.com/ontologies/edamam.owl#recipe_ff0a16e4c1a90be6d05a5a7ad4e80c9a', 'label': 'Baked Camembert with Fresh Fruit', 'image': 'https://edamam-product-images.s3.amazonaws.com/web-img/52c/52c51b7303f56636b600f7380ee8f6b7.jpg?X-Amz-Security-Token=IQoJb3JpZ2luX2VjEMP%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEaCXVzLWVhc3QtMSJGMEQCIBvWDQH2SkxrbLvwYi9Ay7TmDD52X4LgNGBZ9f1zB20kAiBtJYiosEHYbsIaeLyDdXr9e6wUYB5Rhfvcx9ju%2BlMYPirVBAir%2F%2F%2F%2F%2F%2F%2F%2F%2F%2F8BEAAaDDE4NzAxNzE1MDk4NiIMqPRKH

In [23]:
import requests
import json

api_id = '0ceaaf83'
api_key = '93ff26f92c228dbafa3d7d2f9b789a39'

endpoint = 'https://api.edamam.com/api/recipes/v2'

data = requests.get(endpoint, params={'q':'camembert','app_id':api_id,'app_key':api_key,'type':'public'})

In [26]:
print(data.text)

{"from":1,"to":20,"count":449,"_links":{"next":{"href":"https://api.edamam.com/api/recipes/v2?q=camembert&app_key=93ff26f92c228dbafa3d7d2f9b789a39&_cont=CHcVQBtNNQphDmgVQntAEX4BYlRtDQUHQ2JHBWcSalJ0AQAOUXlSVTYTZQQhAVADRjQUBWMUZ1MgBAEOQG0VBWYQYVN6VQMVLnlSVSBMPkd5BgMbUSYRVTdgMgksRlpSAAcRXTVGcV84SU4%3D&type=public&app_id=0ceaaf83","title":"Next page"}},"hits":[{"recipe":{"uri":"http://www.edamam.com/ontologies/edamam.owl#recipe_ff0a16e4c1a90be6d05a5a7ad4e80c9a","label":"Baked Camembert with Fresh Fruit","image":"https://edamam-product-images.s3.amazonaws.com/web-img/52c/52c51b7303f56636b600f7380ee8f6b7.jpg?X-Amz-Security-Token=IQoJb3JpZ2luX2VjEML%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEaCXVzLWVhc3QtMSJHMEUCIAl0YF5JEwWW%2FGv%2FJPTcRILU%2B0cnXxIvNvTCPpPrQ9NAAiEA%2BjKnRq%2F1pow%2FpU6T98YCQTTotTILMyzGhJnv2t%2FsGUQq1QQIq%2F%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FARAAGgwxODcwMTcxNTA5ODYiDAiyY8KIe9u1kliP%2FiqpBHb4INHMRJTRT1FYIVdsilWsW070ewE40Xtzk%2FnABrYmTXXDukZWNv3CJHHMnGzJHY0AweoBY7BCxrqGmlleY2iJTv%2FVB7w4ggJxOBp

In [28]:
js = data.json()

In [29]:
recetas = []
calorias = []

for receta in js['hits']:
    recetas.append(receta['recipe']['label'])
    calorias.append(receta['recipe']['calories'])

In [33]:
pd.DataFrame({'recetas':recetas, 'calorias':calorias}).sort_values('calorias',ascending=False)

,recetas,calorias
4,Baked camembert,5911.433894
2,Puff pastry tarts with broccoli and Camembert ...,3147.970693
10,Sausage-Camembert Spoon Pudding,2550.062302
14,"Potato, Apple, And Camembert Timbales",2457.949532
11,Camembert Crostini,1882.240209
12,Camembert and Wild Mushroom Fondue,1685.041028
16,Camembert and Cranberry Compote Toasts,1684.840000
6,Baked Camembert with Thyme & Garlic,1672.991124
17,Camembert Mashed Potatoes,1449.672001
8,Ham and Camembert Baguette Sandwich recipes,1416.203587
